In [2]:
import torch
import numpy as np
from auto_LiRPA import BoundedModule, BoundedTensor, PerturbationLpNorm
from collections import defaultdict

In [3]:
nn_model = "24_bus_128node.pt"
model = torch.load(nn_model, weights_only=False)
model.eval()

Sequential(
  (0): Linear(in_features=87, out_features=128, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=128, out_features=128, bias=True)
  (5): Sigmoid()
  (6): Linear(in_features=128, out_features=38, bias=True)
)

In [4]:
# relax it!
lb  = -1*torch.ones(87).unsqueeze(0) 
ub  = +2*torch.ones(87).unsqueeze(0) 
ptb = PerturbationLpNorm(x_L=lb, x_U=ub)

# this is the input around which we relax
mod_input     = (lb+ub)/2
bounded_model = BoundedModule(model, mod_input)
mod_input     = BoundedTensor(mod_input, ptb)
lb_al, ub_al  = bounded_model.compute_bounds(x=(mod_input,), method="alpha-CROWN")

In [5]:
for node_name, node in bounded_model.named_modules():
    print(f"Node: {node_name}, Type: {type(node)}")
    # These nodes contain the 'linear' parameters for relaxation


Node: , Type: <class 'auto_LiRPA.bound_general.BoundedModule'>
Node: /0, Type: <class 'auto_LiRPA.operators.leaf.BoundInput'>
Node: /1, Type: <class 'auto_LiRPA.operators.leaf.BoundParams'>
Node: /2, Type: <class 'auto_LiRPA.operators.leaf.BoundParams'>
Node: /3, Type: <class 'auto_LiRPA.operators.leaf.BoundParams'>
Node: /4, Type: <class 'auto_LiRPA.operators.leaf.BoundParams'>
Node: /5, Type: <class 'auto_LiRPA.operators.leaf.BoundParams'>
Node: /6, Type: <class 'auto_LiRPA.operators.leaf.BoundParams'>
Node: /7, Type: <class 'auto_LiRPA.operators.leaf.BoundParams'>
Node: /8, Type: <class 'auto_LiRPA.operators.leaf.BoundParams'>
Node: /9, Type: <class 'auto_LiRPA.operators.linear.BoundLinear'>
Node: /10, Type: <class 'auto_LiRPA.operators.s_shaped.BoundSigmoid'>
Node: /11, Type: <class 'auto_LiRPA.operators.linear.BoundLinear'>
Node: /12, Type: <class 'auto_LiRPA.operators.s_shaped.BoundSigmoid'>
Node: /13, Type: <class 'auto_LiRPA.operators.linear.BoundLinear'>
Node: /14, Type: <clas

In [23]:
required_A = defaultdict(set)
required_A[bounded_model.output_name[0]].add(bounded_model.input_name[0])

lb_al, ub_al, A  = bounded_model.compute_bounds(x=(mod_input,), method="alpha-CROWN", return_A=True, needed_A_dict=required_A)